In [ ]:
from pathlib import Path

import sys
import numpy as np
import pandas as pd
import rasterio
import refet
import pyproj as proj
import xarray
import requests
import multiprocessing as mp

In [ ]:
#sys.path.append("//")
sys.path.append("../../../Micromet")
import micromet
from micromet.volk import ffp_climatology as ffp
from micromet import AmerifluxDataProcessor

In [ ]:
import configparser
# load initial flux data
station = 'US-UTW'
config_path = f'../../station_config/{station}.ini'
config = configparser.ConfigParser()
config.read(config_path)

spath = "../../secrets/config.ini"
sconfig = configparser.ConfigParser()
sconfig.read(spath)

ed_pass = sconfig['NLDAS']['pw']
ed_user = sconfig['NLDAS']['user']

In [ ]:
import datetime

#nldas_out_dir = Path('C:/Users/paulinkenbrandt/Documents/GitHub/MicroMet/Notebooks/Footprints')


for date in pd.date_range(start='2022-01-01', end='2025-12-31',freq='h'):
    hour = date.hour
    if 5 <= hour <= 21:
        micromet.download_nldas(date,
                           hour,
                           ed_user,
                           ed_pass,)